This notebook contains slides for presenting some functionality and implementation details of the Prolog Jupyter kernel.


It was created for a [RISE](https://rise.readthedocs.io/en/stable/index.html) slideshow started from Jupyter Notebook.

# A Jupyter Kernel for Prolog

- Execute queries

In [1]:
atom_length(atom,B)

B = 4

- Define predicates 

In [2]:
my_append([], Res, Res).
my_append([H|T], List, [H|Res]) :-
  my_append(T, List, Res).

% Asserting clauses for user:my_append/3


In [3]:
my_append([1,2], [3,4], R).

R = [1,2,3,4]

- Provides the possibility of executing Prolog code with Jupyter applications
- Replicates the standard Prolog usage and adds convenience functionality


- **Execute queries**: Example showing additional functionality: `atom_length(atom, L)`
    - Exact name? *Tab* &rarr; completion
    - Argument order? *Shift+Tab* &rarr; inspection 
    - Note: **missing terminating full-stop**
        - Eliminate a cause for queries not to be run right away


- Also: **Define predicates**


- Jupyter can be used to create notebooks consisting of cells like these ones
    - Source code and documentation
    - Create Assignments
    - Create slides for lectures like these ones
    - &rarr; Useful for teaching Prolog


- Before presenting implementation details: **General overview** of most important features for SWI-Prolog

## Differentiating Term Types

- Each code cell can contain multiple terms: **clause definitions**, **directives** and **queries**


### Query
- Single term without body in a cell


In [4]:
X = [1,2,3], append(X, [4,5,6], Z).

X = [1,2,3],
Z = [1,2,3,4,5,6]

- Term starting with `?-` or `:-`

In [ ]:
?- format('Hello').
?- member(4, [1, 2, 3]).

Hello

true

false

In [6]:
:- member(M, [x,y,z]).

- Console: type in queries
    - Code cell cannot only contain queries
- Problem: clauses **without** bodies and queries
- The output indicates how a term was interpreted


- Queries with prefixes: Even if the cell contains further terms
- Directives: no result

### Clause definition
- Any other term

In [7]:
fact(a).
fact(b).

% Asserting clauses for user:fact/1


- By default, previous clauses are retracted

In [8]:
fact(c).
fact(d).

Previously defined clauses of user:fact/1 were retracted:
:- dynamic fact/1.

fact(a).
fact(b).

% Asserting clauses for user:fact/1


- Clause are added as **dynamic** facts to the database


- Jupyter applications were developed for **interactive programming**
    - Involves writing, testing and rewriting clauses rather than adding new clauses to the fact database
    - &rarr; By default: retract previous clauses

### Clause definition

- New clauses can be added instead by declaring the predicate `discontiguous`

In [9]:
:- discontiguous disc_fact/1.
:- dynamic disc_fact/1.

In [10]:
disc_fact(a).
disc_fact(b).

% Asserting clauses for user:disc_fact/1


In [11]:
disc_fact(c).
disc_fact(d).

% Asserting clauses for user:disc_fact/1


In [12]:
listing(disc_fact)

:- dynamic disc_fact/1.

disc_fact(a).
disc_fact(b).
disc_fact(c).
disc_fact(d).


true

- A user might want to define a predicate in separate cells

## Handling Multiple Solutions

- Mimicking the usual backtracking mechanism

In [13]:
member(M, [a,b,c]).

M = a

In [14]:
jupyter:retry.

% Retrying goal: member(M,[a,b,c])


M = b

- Problem of the Jupyter kernel: **user intraction** not supported

## Debugging

- Printing the trace of a goal

In [15]:
app([], Res, Res).
app([Head|Tail], List, [Head|Res]) :-
  app(Tail, List, Res).

% Asserting clauses for user:app/3


In [16]:
jupyter:trace(app([1], [2], R)).

   Call: (84) app([1], [2], _51818)
   Call: (85) app([], [2], _52936)
   Exit: (85) app([], [2], [2])
   Exit: (84) app([1], [2], [1, 2])

R = [1,2]

- Debugging cannot be performed interactively
- Instead, print the trace of a goal

## Running Automated Tests

- PlUnit tests can be loaded from a file


In [17]:
?- consult(test).
?- run_tests.

true

% PL-Unit: test ... done
% All 3 tests passed

true

- ... or be defined in a cell

In [18]:
:- begin_tests(list). 

test(list) :-
  lists:is_list([]).

:- end_tests(list).


% Defined test unit list

In [19]:
run_tests.

% PL-Unit: test ... done
% PL-Unit: list . done
% All 4 tests passed

true

## Benchmarking Capabilities

- Whenever a query is executed, its runtime is stored in the database

In [20]:
member(M, [1,2,3]).

M = 1

In [21]:
jupyter:print_query_time.

Query:   member(M,[1,2,3])
Runtime: 0 ms

true

- Access the previous goal and its runtime

## Structured Output

- Display all possible results of a goal in a table

In [22]:
jupyter:print_table((member(Member, [10,20,30,40]), Square is Member*Member)).

Member | Square | 
:- | :- | 
10 | 100 | 
20 | 400 | 
30 | 900 | 
40 | 1600 | 

true

## Introspection

- Code completion: *Tab*
    - For predicates which are **built-in** or **exported** by a loaded module

- Predicate inspection: *Shift + Tab*
    - Help retrieved `help/1`

- Various `jupyter` predicates
    - Access documentation with a help predicate

In [23]:
jupyter:help

jupyter:cut or cut

    Cuts off the choicepoints of the latest active query.

    In general, the previous query is the active one.
    However, the previous active query can be activated again.
    This can be done by cutting off choicepoints with jupyter:cut/0.
    This is also the case if a retry/0 encounters no further solutions.

    A further retry/0 call causes backtracking of the previous active goal.

    Needs to be the only goal of a query.

--------------------------------------------------------------------------------

jupyter:halt or halt

    Shuts down the running Prolog process.

    The next time code is to be executed, a new process is started.
    Everything defined in the database before does not exist anymore.

    Corresponds to the functionality of halt/0.
    Has the same effect as interrupting or restarting the Jupyter kernel.

--------------------------------------------------------------------------------

jupyter:help

    Outputs the documentation for al

true

- Various `juypter` (mostly convenience) predicates
- Difficult to remember all of them
    - In addition to completion and inspection: predicate to print all documentation

## Jupyter

- Originates from the **IPython** project
    - Enables interactive Python development
    - Several frontends, including a former version of **Jupyter Notebook**
        - Web application for handling Jupyter notebooks
        - Planned to be replaced by **JupyterLab**



- *Two-process model*:
<img style="float: right; max-width: 40%;" src="user_interaction_diagram.png">

    - Client process: responsible for user interaction
    - Kernel process: handles code execution

## Architecture

<img style="max-width: 80%;" src="architecture_diagram.png">






Kernel split in three:
- Extends IPython kernel: **inherits** the communication with a frontend via the ZeroMQ protocol


- Does not interpret Prolog itself
    - Starts an existing Prolog instance in a **subprocess**
        - Communicates with it according to the JSON-RPC 2.0 protocol
    - For any code execution **request**:
        - Sends a request message to the Prolog server containing the **code**
        - Prolog terms are read from the code and handled
    
    
- Make the kernel **extensible**: additional layer of a *kernel implementation* in between
    - **Responsible** for basically all functionality (e.g. handling Prolog **server**)
    - For every request the kernel receives, a **method** of the implementation class is called
    - Kernel started: loads **config** file
        - Can contain paths to interpreter-specific Python class files
    - By **extending** default implementation class and **overriding** methods
        - Kernel behaviour can be adjusted
    - Had to be done to support predicate inspection for both    
- Configure to start a different Prolog server

## Changing the Prolog Implementation

- Switch between Prolog implementations on the fly
- The previous server process is kept running
    - When switching back, the database state has not changed

In [24]:
jupyter:set_prolog_impl(sicstus).

true

In [ ]:
app([1,2], [3], R)

! Existence error in user:app/3
! procedure user:app/3 does not exist
! goal:  user:app([1,2],[3],_167773)


In [26]:
jupyter:set_prolog_impl(swi).

yes

In [27]:
app([1,2], [3], R)

R = [1,2,3]

## Extending the Kernel

- At first, the kernel was developed for SICStus Prolog only
    - Adjusted for SWI-Prolog as well
    - Made extensible for further Prolog implementations
    

- By **replacing the Prolog server**, the Python part can easily support a different implementation
    - Requirements: receive requests as JSON-RPC 2.0 messages, handle them, and send responses
    - Might be possible to further extend the existing server with conditional compilation
        - Advanced features might require significant changes

- By **overriding the `PrologKernelBaseImplementation` class**, most of the basic kernel behaviour can be adjusted
    - For SICStus and SWI-Prolog, the handling of predicate inspection differs

- Server replacement:
    - Most code compatible with SICStus and SWI &rarr; **conditional compilation**
    - Expected to be similar for other implementations
    - Extend existing:
        - Implementing **basic code execution** should not require major effort
        - More **advanced features** might involve significant changes


- Replacement of the server does not suffice?: Python extension
- In case of SWI- and SICStus, the only Python code that differs is for predicate inspection

## Future Work

- Support further Prolog implementations
    - Or multiple versions of the same implementation

- Combine strengths of several Prolog instances
    - Kernel can be connected with multiple servers at once
    - Reusing results for another one should be relatively easy

- Send commands to all available Prolog servers *at once*
    - Detect differences in the behaviour
    - Compare the performance by using the benchmarking functionality